In [10]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [11]:
# include_see_also=False used for validation below.
# in deployment, include_see_also should be set to True
gc = GraphCreator("Decision tree", include_see_also=False, max_recursize_requests=50) 
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 258 


['Tree (graph theory)', 'Causal model', 'Decision tree learning', 'Utility', 'Probability', 'Goal', 'Algorithm', 'Decision support system', 'Operations research', 'Decision analysis'] 


['Behavior tree (artificial intelligence, robotics and control)', 'Boosting (machine learning)', 'Decision cycle', 'Decision list', 'Decision table', 'Decision tree model', 'Design rationale', 'DRAKON', 'Markov chain', 'Random forest', 'Odds algorithm', 'Topological combinatorics', 'Truth table']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [12]:
gc.expand_network_threaded(threads=20, chunk_size=1)
print("Number of Links After Expansion: ", len(gc.graph.nodes))

Number of Links After Expansion:  122200


3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [13]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [14]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Hysteria Project 2,Hysteria Project,7
1,Hysteria Project,Hysteria Project 2,7
2,Visual analytics,Visualization (graphics),5
3,Concept map,Mind map,5
4,Deep learning,Glossary of artificial intelligence,5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [15]:
features_df = gc.get_features_df(rank=False)

# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [16]:
gc.rank_similarity()
gc.features_df.sort_values("similarity_rank", ascending=False).head(10)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
85,Decision tree,256,1,170,86,1.000000,0.066912,0.000345,0.020580,0.0,0.0,1.000000,0,7.897503
124,Decision analysis,119,1,70,49,0.065147,0.008614,0.000117,0.008474,1.0,1.0,0.038961,1,1.854459
67,Decision tree learning,328,1,172,156,0.067285,0.063108,0.000276,0.054475,1.0,1.0,0.052308,1,1.632265
163,Influence diagram,54,1,20,34,0.063241,0.012019,0.000056,0.004237,1.0,1.0,0.038251,0,1.022532
261,"Behavior tree (artificial intelligence, roboti...",25,1,6,19,0.016736,0.011611,0.000021,0.002421,1.0,1.0,0.017341,0,1.004848
162,Information gain in decision trees,54,1,11,43,0.042636,0.010239,0.000022,0.004237,1.0,1.0,0.028409,0,1.001297
145,ID3 algorithm,80,1,20,60,0.060932,0.010530,0.000033,0.003632,1.0,1.0,0.043956,0,0.993084
164,Decision table,53,1,31,22,0.034749,0.004408,0.000136,0.004237,1.0,1.0,0.020305,0,0.989380
182,Decision tree model,39,1,14,25,0.019920,0.005143,0.000037,0.003026,1.0,1.0,0.022222,0,0.988322
73,Random forest,292,1,160,132,0.055825,0.071310,0.000268,0.047817,1.0,1.0,0.050955,0,0.902944


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [17]:
scaled_feature_df = gc.scale_features_df(scaler=MinMaxScaler, copy=True) # Makes a copy of the df
scaled_feature_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Decision tree,0.009938,1.0,0.006769,0.028467,1.000000,2.368394e-01,6.283811e-03,0.062615,0.000,0.0,1.000000,0.0,1.000000e+00
1,Decision analysis,0.004599,1.0,0.002787,0.016220,0.065147,3.048976e-02,2.112196e-03,0.025783,0.125,0.2,0.038961,1.0,2.348158e-01
2,Decision tree learning,0.012744,1.0,0.006849,0.051639,0.067285,2.233751e-01,5.019837e-03,0.165746,0.125,0.2,0.052308,1.0,2.066811e-01
3,Influence diagram,0.002066,1.0,0.000796,0.011255,0.063241,4.254281e-02,9.940081e-04,0.012891,0.125,0.2,0.038251,0.0,1.294753e-01
4,"Behavior tree (artificial intelligence, roboti...",0.000935,1.0,0.000239,0.006289,0.016736,4.109937e-02,3.540656e-04,0.007366,0.125,0.2,0.017341,0.0,1.272361e-01
5,Information gain in decision trees,0.002066,1.0,0.000438,0.014234,0.042636,3.624161e-02,3.653633e-04,0.012891,0.125,0.2,0.028409,0.0,1.267864e-01
6,ID3 algorithm,0.003079,1.0,0.000796,0.019861,0.060932,3.727311e-02,5.703136e-04,0.011050,0.125,0.2,0.043956,0.0,1.257465e-01
7,Decision table,0.002027,1.0,0.001234,0.007282,0.034749,1.560405e-02,2.450502e-03,0.012891,0.125,0.2,0.020305,0.0,1.252775e-01
8,Decision tree model,0.001481,1.0,0.000557,0.008275,0.019920,1.820236e-02,6.316382e-04,0.009208,0.125,0.2,0.022222,0.0,1.251435e-01
9,Random forest,0.011341,1.0,0.006371,0.043694,0.055825,2.524059e-01,4.862491e-03,0.145488,0.125,0.2,0.050955,0.0,1.143327e-01


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [18]:
evaluate_metrics(scaled_feature_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.969235,0.999894,0.030658,13.0,0.846154,1.000000,1.000000,1.0
centrality,0.945918,0.999894,0.053975,13.0,0.615385,0.923077,1.000000,1.0
adjusted_reciprocity,0.944124,0.999894,0.055769,13.0,0.923077,0.923077,1.000000,1.0
page_rank,0.977052,0.999894,0.022842,13.0,0.692308,1.000000,1.000000,1.0
shortest_path_length_from_entry,0.000041,0.999894,0.999853,13.0,0.000000,0.000000,0.000000,0.0
jaccard_similarity,0.841696,0.999894,0.158198,13.0,0.384615,0.846154,0.846154,1.0


The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___